In [1]:
import os
import time
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import Ws_model_generator as wsg

In [2]:
def file_reader(file_path):
    with open(file_path) as f:
        array_neigh = [x for x in f.read().split('\n')]
        # print(len(array_neigh))
    return array_neigh

In [3]:
def myformula_for_finding_n_u_1(n, k):
    A = int(np.floor((n - 1)/k))
    n_v = [0] * (int(k/2))
    h = int(k/2)
    mod_rem = np.mod(n + 1, k)
    rem_val = [the_rem for the_rem in range(0, int(k/2))]
    for i in range(int(k/2)):
        if mod_rem not in rem_val[:i+1]:
            n_v[i] = (i+1) * A + 1
        else:
            n_v[i] = (i+1) * A + 1 - (i + 1 - mod_rem)
    return n_v

In [4]:
def for_index_finding(n_u, n_v, edge_vert = [1, 1], ws_indices=[1, 1, 1, 1, 1]):
    if len(n_u) != len(n_v):
        print("The length of n_u and n_v are not equal")
        return 0, 0, 0, 0, 0
    else:
        product_val = n_u * n_v
        sum_val = n_u + n_v
        sz_index = np.sum(product_val)/ (ws_indices[0] * edge_vert[1])
        PI_index = np.sum(sum_val)/ (ws_indices[1] * edge_vert[1])
        ABC_index = np.sum(np.sqrt((sum_val - 2) / product_val))/ (ws_indices[2])
        NGG_index = (np.sum(1/np.sqrt(product_val))) / (ws_indices[3])
        Mo_index = np.sum(np.abs(n_u - n_v)) / edge_vert[1]
        return sz_index, PI_index, ABC_index, NGG_index, Mo_index

In [5]:
def index_creater(folder_path):
    edges_vertices_path = os.path.join(folder_path, 'edges_vertices.txt')
    neigh_u_path = os.path.join(folder_path, 'neigh_u_val.txt')
    neigh_v_path = os.path.join(folder_path, 'neigh_v_val.txt')
    edges_vertices = file_reader(edges_vertices_path)
    neigh_u = file_reader(neigh_u_path)
    neigh_v = file_reader(neigh_v_path)
    indices = []
    for i in range(len(neigh_u) - 1):
        try:
            val1 = np.array([int(x) for x in neigh_u[i][1:-1].split(",")])
            val2 = np.array([int(x) for x in neigh_v[i][1:-1].split(",")])  
            val3 = np.array([int(x) for x in edges_vertices[i][1:-1].split(",")])      
        except:
            print("Error in line: ", i)  
            break 
        k_values = int(val3[1]/val3[0])
        n_u = myformula_for_finding_n_u_1(val3[0], k_values)
        n_u = n_u * val3[0]
        n_u = np.array(n_u)
        ws_index = for_index_finding(n_u, n_u)
        normalised_index = for_index_finding(val1, val2, ws_index)
        indices.append(normalised_index)
    return indices

In [62]:
root_folder = 'Alzheimers_graph_valued_Dataset_2'
for folder_path, _, files in os.walk(root_folder):
    if folder_path.endswith('Demented'):
        index_arrays = index_creater(folder_path)
        df = pd.DataFrame(index_arrays, columns=["sz_index", "PI_index", "ABC_index", "NGG_index", "Mo_index"])
        df.to_csv(os.path.join(folder_path, 'indices_normalised.csv'))
        print("Done with folder: ", folder_path)
        continue

Done with folder:  Alzheimers_graph_valued_Dataset_2/test/VeryMildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/ModerateDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/MildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/NonDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/train/VeryMildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/train/ModerateDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/train/MildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/train/NonDemented


In [12]:
Dementia_class = {'NonDemented': 0, 'ModerateDemented': 3, 'VeryMildDemented': 1, 'MildDemented': 2}
root_folder = 'Alzheimers_graph_valued_Dataset_2/test/'
df = pd.DataFrame(columns=["sz_index", "PI_index", "ABC_index", "NGG_index", "Mo_index", "Dementia_class"])
for folder_path, _, files in os.walk(root_folder):
    if folder_path.endswith('Demented'):
        index_arrays_val = pd.read_csv(os.path.join(folder_path, 'indices_normalised.csv'))
        index_arrays_val = index_arrays_val.drop(['Unnamed: 0'], axis=1)
        index_arrays_val['Dementia_class'] = Dementia_class[folder_path.split('/')[-1]]
        print("Done with folder: ", folder_path)
        df = pd.concat([df, index_arrays_val], ignore_index=True)
        continue        

Done with folder:  Alzheimers_graph_valued_Dataset_2/test/VeryMildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/ModerateDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/MildDemented
Done with folder:  Alzheimers_graph_valued_Dataset_2/test/NonDemented


In [13]:
df.to_csv('Alzheimers_graph_valued_Dataset_2/test/normalised_indices.csv')